In [9]:
pip install duckduckgo_search youtube-search-python google-play-scraper

In [10]:
pip install ddgs youtube-search-python google-play-scraper

In [11]:
import sys
import json
from datetime import datetime
from ddgs import DDGS  # Updated to use ddgs library
from youtube_search import YoutubeSearch  # For YouTube search (scraping-based)
import re

# For app store, ensure google-play-scraper is installed
try:
    from google_play_scraper import search as gp_search
except ImportError:
    gp_search = None

# Detect if running in Jupyter/IPython
try:
    from IPython import get_ipython
    in_ipython = get_ipython() is not None
except ImportError:
    in_ipython = False

def search_web(topic):
    """
    Searches the general web for the given topic using DDGS with exact phrase.
    Returns a list of results with titles, snippets, and links.
    """
    print(f"Searching the general web (Google-like) for exact: \"{topic}\"")
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(f'"{topic}"', max_results=50)]
            return [{
                'source': 'web',
                'title': r.get('title', ''),
                'snippet': r.get('body', ''),
                'link': r.get('href', ''),
                'timestamp': datetime.now().isoformat()
            } for r in results]
    except Exception as e:
        print(f"Web search error: {e}")
        return []

def search_news(topic):
    """
    Searches news sources for the given topic using DDGS news API with exact phrase.
    """
    print(f"Searching news for exact: \"{topic}\"")
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.news(f'"{topic}"', max_results=50)]
            return [{
                'source': 'news',
                'title': r.get('title', ''),
                'snippet': r.get('body', ''),
                'link': r.get('url', ''),
                'timestamp': r.get('date', datetime.now().isoformat())
            } for r in results]
    except Exception as e:
        print(f"News search error: {e}")
        return []

def search_youtube(topic):
    """
    Searches YouTube for videos related to the topic with exact phrase.
    Uses youtube-search library (scraping-based; be mindful of YouTube's TOS).
    """
    print(f"Searching YouTube for exact: \"{topic}\"")
    try:
        results = YoutubeSearch(f'"{topic}"', max_results=50).to_dict()
        return [{
            'source': 'youtube',
            'title': r.get('title', ''),
            'views': r.get('views', ''),
            'duration': r.get('duration', ''),
            'link': f"https://www.youtube.com{r.get('url_suffix', '')}",
            'timestamp': r.get('publish_time', datetime.now().isoformat()),
            'snippet': r.get('long_desc', '') or r.get('description', '')
        } for r in results]
    except Exception as e:
        print(f"YouTube search error: {e}")
        return []

def search_social(topic, platform):
    """
    Searches a specific social platform (e.g., Instagram, Facebook) using site-specific web search with exact phrase.
    Note: Results limited to public content indexed by search engines.
    """
    site = f"{platform}.com"
    print(f"Searching {platform} for exact: \"{topic}\" (via site-specific search)")
    try:
        with DDGS() as ddgs:
            query = f'"{topic}" site:{site}'
            results = [r for r in ddgs.text(query, max_results=50)]
            return [{
                'source': platform,
                'title': r.get('title', ''),
                'snippet': r.get('body', ''),
                'link': r.get('href', ''),
                'timestamp': datetime.now().isoformat()
            } for r in results]
    except Exception as e:
        print(f"{platform} search error: {e}")
        return []

def search_app_stores(topic):
    """
    Searches Google Play Store for apps related to the topic.
    Requires google-play-scraper installed.
    """
    if gp_search is None:
        print("google-play-scraper not installed. Install with: pip install google-play-scraper")
        return []

    print(f"Searching app stores (Google Play) for: \"{topic}\"")
    try:
        apps = gp_search(topic, lang="en", country="us")
        return [{
            'source': 'app_store',
            'app_name': app.get('title', ''),
            'description': app.get('description', ''),
            'link': f"https://play.google.com/store/apps/details?id={app.get('appId', '')}",
            'timestamp': datetime.now().isoformat()
        } for app in apps[:50]]
    except Exception as e:
        print(f"App store search error: {e}")
        return []

def has_exact_match(result, query):
    """
    Checks if the query sentence appears exactly (case-insensitive) in the result's relevant fields.
    Handles None values safely.
    """
    query_lower = query.lower()
    fields = [
        result.get('title', '') or '',
        result.get('snippet', '') or '',
        result.get('description', '') or '',
        result.get('views', '') or '',
        result.get('long_desc', '') or ''
    ]
    return any(query_lower in (field.lower() if isinstance(field, str) else '') for field in fields)

def search_all(topic):
    """
    Combines searches from web, news, YouTube, Instagram, Facebook, and app stores.
    Filters for 100% exact matches to the input sentence.
    Saves results to a JSON file for quick retrieval.
    """
    all_results = []

    # General web search
    web_results = search_web(topic)
    all_results.extend(web_results)

    # News search
    news_results = search_news(topic)
    all_results.extend(news_results)

    # YouTube search
    youtube_results = search_youtube(topic)
    all_results.extend(youtube_results)

    # Facebook search (site-specific)
    facebook_results = search_social(topic, 'facebook')
    all_results.extend(facebook_results)

    # Instagram search (site-specific)
    instagram_results = search_social(topic, 'instagram')
    all_results.extend(instagram_results)

    # App store search
    app_results = search_app_stores(topic)
    all_results.extend(app_results)

    # Filter for exact matches
    filtered_results = [r for r in all_results if has_exact_match(r, topic)]

    # Sanitize topic for safe filename
    safe_topic = re.sub(r'[\\/:*?"<>|]', '_', topic.replace(' ', '_'))

    # Save results to JSON file
    output_file = f"search_results_{safe_topic}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_results, f, indent=4)

    print(f"\nFound {len(filtered_results)} exact match results. Saved to {output_file}")
    return filtered_results

def print_results(results):
    """
    Prints the search results with direct links in a readable format.
    """
    for idx, result in enumerate(results, 1):
        print(f"\n{idx}. Source: {result['source']}")
        if result['source'] in ['web', 'news', 'facebook', 'instagram']:
            print(f"   Title: {result['title']}")
            print(f"   Snippet: {result['snippet']}")
            print(f"   Direct Link: {result['link']}")
        elif result['source'] == 'youtube':
            print(f"   Title: {result['title']}")
            print(f"   Views: {result['views']}")
            print(f"   Duration: {result['duration']}")
            print(f"   Snippet: {result['snippet']}")
            print(f"   Direct Link: {result['link']}")
        elif result['source'] == 'app_store':
            print(f"   App Name: {result['app_name']}")
            print(f"   Description: {result['description']}")
            print(f"   Direct Link: {result['link']}")
        print(f"   Timestamp: {result['timestamp']}")

if __name__ == "__main__":
    if in_ipython:
        topic = input("Enter the topic to search: ")
    else:
        if len(sys.argv) > 1:
            topic = " ".join(sys.argv[1:])
        else:
            topic = input("Enter the topic to search: ")

    results = search_all(topic)
    print_results(results)

Enter the topic to search: nuclear war
Searching the general web (Google-like) for exact: "nuclear war"
Searching news for exact: "nuclear war"
Searching YouTube for exact: "nuclear war"
Searching facebook for exact: "nuclear war" (via site-specific search)
Searching instagram for exact: "nuclear war" (via site-specific search)
Searching app stores (Google Play) for: "nuclear war"

Found 165 exact match results. Saved to search_results_nuclear_war_20250815_165635.json

1. Source: web
   Title: Nuclear warfare
   Snippet: Nuclear warfare, also known as atomic warfare, is a military conflict or prepared political strategy that deploys nuclear weaponry. Nuclear weapons are weapons of mass destruction; in contrast to conventional warfare, nuclear warfare can produce destruction in a much shorter time and can have a long-lasting radiological result. A major nuclear exchange would likely have long-term effects, primarily from the fallout released, and could also lead to secondary effects, su